In [19]:
import requests
import re
import json
from lxml import etree
headers = {
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36'
}
url ='https://www.bilibili.com/bangumi/play/ss1559?spm_id_from=333.337.0.0'
resp = requests.get(url, headers=headers)
print(resp.status_code)
text = resp.content.decode('utf-8')
result = re.findall(r'window.__INITIAL_STATE__=(.*?);', text)
comment_data = {}
if result:
    data = json.loads(result[0])
    epList = data.get('epList', [])
    for item in epList:
        cid = item.get('cid')
        comment_data[cid] = []
        url = f"https://comment.bilibili.com/{cid}.xml"
        res = requests.get(url)
        xml_data = res.content
        with open(f'./{cid}.xml', 'wb') as f:
            f.write(xml_data)
        soup = etree.HTML(xml_data)
        content_all = soup.xpath('//d')
        for content in content_all:
            data = content.xpath('string(.)')
            p = content.xpath('./@p')
            if not p:
                continue
            p = p[0]
            bullet_chat_time, _type, size, color, send_time, pool, uid, row_id, other = p.split(',')
            data = content.xpath('string(.)')
            comment_data[cid].append({
                "bullet_chat_time": bullet_chat_time,
                "type": _type,
                "size": size,
                "color": color,
                "send_time": send_time,
                "pool": pool,
                "uid": uid,
                "row_id": row_id,
                "other": other,
                "message": data
            })
            
with open('foodwars', 'w', encoding='utf-8') as f:
    f.write(json.dumps(comment_data, indent=4, ensure_ascii=False))


200
